In [ ]:
import pickle
import os
import pandas as pd
import numpy as np

def load_and_prepare_data(spots_file, edges_file, tracks_file):
    print(f"Loading data from {os.path.dirname(spots_file)}")
    
    # Load data
    spot_table = pd.read_csv(spots_file, header=0, skiprows=[1, 2])
    edge_table = pd.read_csv(edges_file, header=0, skiprows=[1, 2])
    track_table = pd.read_csv(tracks_file, header=0, skiprows=[1, 2])
    
    # Convert non-numeric columns to numeric
    non_numeric_cols = ['LABEL']
    for table in [spot_table, edge_table, track_table]:
        numeric_cols = table.columns.difference(non_numeric_cols)
        table[numeric_cols] = table[numeric_cols].apply(pd.to_numeric, errors='coerce')
        table.fillna(0, inplace=True)
    
    return spot_table, edge_table, track_table

def preprocess_track_table_with_time_offset(track_table, time_offset):
    # Convert time from seconds to hours
    track_table['TRACK_START_HOURS'] = track_table['TRACK_START'] / 3600
    track_table['TRACK_STOP_HOURS'] = track_table['TRACK_STOP'] / 3600
    track_table['TRACK_DURATION_HOURS'] = track_table['TRACK_DURATION'] / 3600
    
    # Apply the time offset after converting to hours
    track_table['TRACK_START_HOURS'] += time_offset
    track_table['TRACK_STOP_HOURS'] += time_offset
    
    return track_table

def preprocess_edge_table_with_dynamic_center(spot_table, edge_table, time_offset):
    # Calculate the centroid (geometric center) based on the spot positions
    center_x = spot_table['POSITION_X'].mean()
    center_y = spot_table['POSITION_Y'].mean()
    
    print(f"Dynamic Center: ({center_x:.2f}, {center_y:.2f})")

    # Initialize lists to store computed values
    u_values, v_values = [], []
    source_x_values, source_y_values = [], []
    target_x_values, target_y_values = [], []
    radial_distances, radial_velocities = [], []
    radial_velocity_thetas = []
    centers_x, centers_y = [], []

    for _, row in edge_table.iterrows():
        # Get source and target IDs
        source_id, target_id = row['SPOT_SOURCE_ID'], row['SPOT_TARGET_ID']
        
        # Find corresponding rows in the spot data
        source_spot = spot_table[spot_table['ID'] == source_id]
        target_spot = spot_table[spot_table['ID'] == target_id]
        
        # Check if source_spot or target_spot is empty and handle accordingly
        if source_spot.empty or target_spot.empty:
            continue  # Skip this iteration if either is empty
        
        # Extract source and target positions
        source_x, source_y = source_spot['POSITION_X'].values[0], source_spot['POSITION_Y'].values[0]
        target_x, target_y = target_spot['POSITION_X'].values[0], target_spot['POSITION_Y'].values[0]
        
        # Calculate u and v (movement vectors)
        u, v = target_x - source_x, target_y - source_y
        
        # Calculate the radial distance from source to the center
        r_source = np.sqrt((source_x - center_x) ** 2 + (source_y - center_y) ** 2)
        
        if r_source != 0:
            # Radial direction vector
            radial_direction_x = (center_x - source_x) / r_source
            radial_direction_y = (center_y - source_y) / r_source
            
            radial_direction = np.arctan2(radial_direction_y, radial_direction_x)
            
            # Radial velocity component
            radial_velocity = u * radial_direction_x + v * radial_direction_y
        else:
            # Define default values when r_source is zero
            radial_direction = 0
            radial_velocity = 0
        
        # Store calculated values
        u_values.append(u)
        v_values.append(v)
        source_x_values.append(source_x)
        source_y_values.append(source_y)
        target_x_values.append(target_x)
        target_y_values.append(target_y)
        radial_distances.append(r_source)
        radial_velocities.append(radial_velocity)
        radial_velocity_thetas.append(radial_direction)
        centers_x.append(center_x)
        centers_y.append(center_y)

    # Add calculated values to edge data DataFrame
    edge_table['u'] = u_values
    edge_table['v'] = v_values
    edge_table['source_x'] = source_x_values
    edge_table['source_y'] = source_y_values
    edge_table['target_x'] = target_x_values
    edge_table['target_y'] = target_y_values
    edge_table['radial_distance'] = radial_distances
    edge_table['radial_velocity'] = radial_velocities
    edge_table['center_x'] = centers_x
    edge_table['center_y'] = centers_y
    edge_table['radial_velocity_theta'] = radial_velocity_thetas

    # Convert time from seconds to hours and apply any necessary offset
    edge_table['EDGE_TIME'] = (edge_table['EDGE_TIME'] / 3600) + time_offset
    
    return edge_table

def calculate_radial_persistence(edge_table):
    velocity_magnitude = np.sqrt(edge_table['u']**2 + edge_table['v']**2)
    radial_persistence = np.where(
        velocity_magnitude != 0,
        edge_table['radial_velocity'] / velocity_magnitude,
        0
    )
    return radial_persistence

def load_multiple_data_with_dynamic_center(working_directory, time_offset):
    control_spots_dir = os.path.join(working_directory, 'control_spots')
    control_edges_dir = os.path.join(working_directory, 'control_edges')
    control_tracks_dir = os.path.join(working_directory, 'control_tracks')
    treatment_spots_dir = os.path.join(working_directory, 'treatment_spots')
    treatment_edges_dir = os.path.join(working_directory, 'treatment_edges')
    treatment_tracks_dir = os.path.join(working_directory, 'treatment_tracks')
    
    control_spot_tables, control_edge_tables, control_track_tables = [], [], []
    treatment_spot_tables, treatment_edge_tables, treatment_track_tables = [], [], []

    for file in os.listdir(control_spots_dir):
        if file.endswith('.csv'):
            spot_file = os.path.join(control_spots_dir, file)
            edge_file = os.path.join(control_edges_dir, file.replace('spots', 'edges'))
            track_file = os.path.join(control_tracks_dir, file.replace('spots', 'tracks'))
            spot_table, edge_table, track_table = load_and_prepare_data(spot_file, edge_file, track_file)
            spot_table['Group'] = 'Control'
            edge_table['Group'] = 'Control'
            track_table['Group'] = 'Control'
            edge_table = preprocess_edge_table_with_dynamic_center(spot_table, edge_table, time_offset)
            track_table = preprocess_track_table_with_time_offset(track_table, time_offset)
            control_spot_tables.append(spot_table)
            control_edge_tables.append(edge_table)
            control_track_tables.append(track_table)

    for file in os.listdir(treatment_spots_dir):
        if file.endswith('.csv'):
            spot_file = os.path.join(treatment_spots_dir, file)
            edge_file = os.path.join(treatment_edges_dir, file.replace('spots', 'edges'))
            track_file = os.path.join(treatment_tracks_dir, file.replace('spots', 'tracks'))
            spot_table, edge_table, track_table = load_and_prepare_data(spot_file, edge_file, track_file)
            spot_table['Group'] = 'Treatment'
            edge_table['Group'] = 'Treatment'
            track_table['Group'] = 'Treatment'
            edge_table = preprocess_edge_table_with_dynamic_center(spot_table, edge_table, time_offset)
            track_table = preprocess_track_table_with_time_offset(track_table, time_offset)
            treatment_spot_tables.append(spot_table)
            treatment_edge_tables.append(edge_table)
            treatment_track_tables.append(track_table)

    combined_spots = pd.concat(control_spot_tables + treatment_spot_tables, ignore_index=True)
    combined_edges = pd.concat(control_edge_tables + treatment_edge_tables, ignore_index=True)
    combined_tracks = pd.concat(control_track_tables + treatment_track_tables, ignore_index=True)
    return combined_spots, combined_edges, combined_tracks

# Main processing loop
root_directory = '/your/directory/root'
offset_dirs = ['offset_24', 'offset_27', 'offset_29']
all_combined_spots, all_combined_edges, all_combined_tracks = [], [], []

for offset_dir in offset_dirs:
    time_offset = int(offset_dir.split('_')[1])
    working_directory = os.path.join(root_directory, offset_dir)
    combined_spots, combined_edges, combined_tracks = load_multiple_data_with_dynamic_center(working_directory, time_offset)
    all_combined_spots.append(combined_spots)
    all_combined_edges.append(combined_edges)
    all_combined_tracks.append(combined_tracks)

final_combined_spots = pd.concat(all_combined_spots, ignore_index=True)
final_combined_edges = pd.concat(all_combined_edges, ignore_index=True)
final_combined_tracks = pd.concat(all_combined_tracks, ignore_index=True)

save_directory = '/your/directory/save'
os.makedirs(save_directory, exist_ok=True)

with open(os.path.join(save_directory, 'spots.pkl'), 'wb') as f:
    pickle.dump(final_combined_spots, f)

with open(os.path.join(save_directory, 'edges.pkl'), 'wb') as f:
    pickle.dump(final_combined_edges, f)

with open(os.path.join(save_directory, 'tracks.pkl'), 'wb') as f:
    pickle.dump(final_combined_tracks, f)

print("Processed data has been saved successfully.")
